In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plot
import matplotlib.pyplot as plt
import seaborn as sns



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Goal
 Predict total sales for every product and store in the next month

# Load dataset

In [ ]:
#sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
#test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
#sample_submission.csv - a sample submission file in the correct format.
#items.csv - supplemental information about the items/products.
#item_categories.csv  - supplemental information about the items categories.
#shops.csv- supplemental information about the shops.

#Converted dataset (English)
df_item_en=pd.read_csv('/kaggle/input/english-converted-datasets/items.csv')
df_submission_en=pd.read_csv('/kaggle/input/english-converted-datasets/sample_submission.csv')
df_item_cat_en=pd.read_csv('/kaggle/input/english-converted-datasets/item_categories.csv')
df_sale_train_en=pd.read_csv('/kaggle/input/english-converted-datasets/sales_train.csv')
df_shop_en=pd.read_csv('/kaggle/input/english-converted-datasets/shops.csv')
df_test_en=pd.read_csv('/kaggle/input/english-converted-datasets/test.csv')
#Competition dataset (Russian)
df_item=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
df_item_cat=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_sale_train=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_shop=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
print(df_item.columns)
print(df_submission.columns)
print(df_item_cat.columns)
print(df_sale_train.columns)
print(df_shop.columns)
print(df_test.columns)

print(df_item.dtypes)
print(df_submission.dtypes)
print(df_item_cat.dtypes)
print(df_sale_train.dtypes)
print(df_shop.dtypes)
print(df_test.dtypes)

# Datasets size

In [ ]:
print('items En ' +str(len(df_item_en)))
print('submission En ' +str(len(df_submission_en)))
print('items category En ' +str(len(df_item_cat_en)))
print('sales train En ' +str(len(df_sale_train_en)))
print('shops En ' +str(len(df_shop_en)))
print('tests En ' +str(len(df_test_en)))
print ('----------------------------------------')
print('items ' +str(len(df_item)))
print('submission ' +str(len(df_submission)))
print('items category ' +str(len(df_item_cat)))
print('sales train ' +str(len(df_sale_train)))
print('shops ' +str(len(df_shop)))
print('tests ' +str(len(df_test)))

In [ ]:
print(df_test.groupby('item_id').count())
print(df_sale_train.groupby('item_id').count())

In [ ]:
df_test_item = set(df_test.item_id.unique())
df_train_item = set(df_sale_train.item_id.unique())
print(len(df_test_item))
print(len(df_train_item))

print(df_sale_train.columns)

In [ ]:
df_test_shop = set(df_test.shop_id.unique())
df_train_shop = set(df_sale_train.shop_id.unique())
print(len(df_test_shop))
print(len(df_train_shop))

## Data Wrangling

In [ ]:
df_item.head(1)
print(df_test.columns)
print(df_sale_train.columns)
print(df_submission.columns)
#extra information datasets
print(df_item.columns)
print(df_item_cat.columns)
print(df_shop.columns)

In [ ]:
print(df_test.head(1))
print(df_sale_train.head(1))
print(df_submission.head(1))

In [ ]:
#data merge
df_item_cat_all = df_item.merge(df_item_cat, left_on='item_category_id', right_on='item_category_id')
df_train_item = df_sale_train.merge(df_item_cat_all, left_on='item_id', right_on='item_id')
df_train = df_train_item.merge(df_shop, left_on='shop_id', right_on='shop_id')


In [ ]:
#investigate the sales train dataset
#convert the date fiedl from string to datetime
df_train['date'] = pd.to_datetime(df_train['date'])
print(df_train.date.head(10))
print(df_train.date.value_counts().head(10))
#get train dates sorted
train_dates = df_train.date.value_counts()
train_dates = train_dates.sort_index()
print(train_dates.head(10))

In [ ]:
#plot dates
train_dates.plot(kind="line")
plt.show()

In [ ]:
#get saisonality per month
month = df_train.date.dt.month
year = df_train.date.dt.year
dates = df_train.date

monthly_data = pd.concat([year,month,dates], axis=1)
monthly_data.columns = ['year','month','dates']
monthly_data = monthly_data.groupby(['year','month','dates']).size()
monthly_data = monthly_data.reset_index()
monthly_data = monthly_data.rename(columns={0 :'size'})
monthly_data = monthly_data.groupby(['year','month']).mean()['size']
print(monthly_data.head(10))
monthly_data = monthly_data.unstack(level=1)
print(monthly_data)

In [ ]:
import seaborn as sns
sns.heatmap(monthly_data)
plt.show()

In [ ]:
### get saisonality per month
month = df_train.date.dt.month
year = df_train.date.dt.year
dates = df_train.date
item  = df_train.item_id
shop = df_train.shop_id
sales = df_train.item_cnt_day *  df_train.item_price
category = df_train.item_category_id


monthly_data_new = pd.concat([year,month,dates,item,category,shop,sales], axis=1)
monthly_data_new.columns = ['year','month','dates','item','category','shop','sales']
monthly_data_new.head(3)
#understand saisonality - are they trends for sales during months ?
m = monthly_data_new.groupby(monthly_data_new.month)['sales'].mean()
m.plot(kind='line')
plt.show()



In [ ]:
#get information by shops / year / months
monthly_shop_sales = monthly_data_new.copy()
monthly_shop_sales = monthly_shop_sales.reset_index()
monthly_shop_sales = monthly_shop_sales.groupby(['shop','month']).mean()['sales']
print(monthly_shop_sales.head(10))
monthly_shop_sales = monthly_shop_sales.unstack(level=0)
print(monthly_shop_sales)

In [ ]:
sns.heatmap(monthly_shop_sales)
plt.show()

In [ ]:
#check the trend on quarter (jan-feb-mar), (apr-mai-jun) - (jul-aug-sep) - (oct-nov-dec)
# data re-sampled based on each month
month_sales = monthly_data_new.resample('MS', on='dates').sales.sum()
quarter_sales = monthly_data_new.resample('Q', on='dates').sales.sum()


# aggregating multiple fields for each motnh
monthly_data_new.resample('Q', on='dates').agg({'sales':'sum', 'item':'count','shop':'nunique'})

In [ ]:
# Grouping data based on month and store type
monthly_data_new.groupby([pd.Grouper(key='dates', freq='M'), 'shop']).sales.sum().head(15)

# grouping data and named aggregation on item, item_count, and sales
monthly_data_new.groupby([pd.Grouper(key='dates', freq='M'), 'item']).agg(unique_items=('item', 'nunique'),
         total_quantity=('item','count'),
         total_amount=('sales','sum'))



In [ ]:
#let's normalize by total amount per month for each item and shop
month_sale_pct = monthly_data_new.groupby(monthly_data_new.dates).apply(lambda x: x /x.sum())
print(month_sale_pct)

# Data Transformation

In [ ]:
#Plot data

trend = monthly_data_new.groupby(monthly_data_new.dates)['sales'].mean()
trend.plot(kind='line')
plt.show()


In [ ]:
#found min max of each shop - get month count from min date to max date
monthly_data_new.groupby(['shop']).agg({'dates': [np.min,np.max]})

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

#plot_pacf(monthly_data_new['sales']);
#plot_acf(monthly_data_new['sales']);

In [ ]:
### month_sales = monthly_shop_sales.groupby(['item_id','month']).mean()['dates','sales']
#monthly_data_new['item'].groupby(monthly_data_new.dates)['sales'].mean()
#monthly_shop_sales.head(5)
data = monthly_data_new[["dates", "item","sales"]]
y = data.set_index(['dates'])
y.sort_index(inplace=True)
y.index.name = 'datetimeindex'
y_ex = np.column_stack((y['item'].values,y['sales'].values))

y_ex




In [ ]:
y.plot(figsize=(19, 4))
plt.show()

In [ ]:
import statsmodels.api as sm
from pylab import rcParams
period = 4
#Decompose the time-series into three distinct components: trend, seasonality, and noise
result_add = sm.tsa.seasonal_decompose(x=data['sales'], model='additive', extrapolate_trend='freq', period=period)
plt.rcParams.update({'figure.figsize': (5,5)})
result_add.plot().suptitle('Additive Decompose', fontsize=22)
plt.show()

**Explaining the model**

**SARIMA** is used for non-stationary series, that is, where the data do not fluctuate around the same mean, variance and co-variance. This model can identify trend and seasonality, which makes it so important. The SARIMA consists of other forecasting models:

**AR**: Auto regressive model (can be a simple, multiple or non-linear regression)

**MA**: Moving averages model. The moving average models can use weighting factors, where the observations are weighted by a trim factor (for the oldest data in the series) and with a higher weight for the most recent observations.

-------
Seasonal Elements

There are four seasonal elements that are not part of ARIMA that must be configured; they are:

    P: Seasonal autoregressive order.
    D: Seasonal difference order.
    Q: Seasonal moving average order.
    m: The number of time steps for a single seasonal period.

A P=1 would make use of the first seasonally offset observation in the model, e.g. t-(m*1) or t-12. A P=2, would use the last two seasonally offset observations t-(m * 1), t-(m * 2).

Similarly, a D of 1 would calculate a first order seasonal difference and a Q=1 would use a first order errors in the model (e.g. moving average).

In [ ]:
import warnings
import itertools
warnings.filterwarnings("ignore")
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y["sales"],order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y["sales"],
                                order=(0, 0, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
ad_fuller_result = adfuller(y['sales'])
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')


In [ ]:
#get the difference of sales compare to the previous month
df_sales_previous = df_sale_train.copy()
df_sales_previous['diff'] = df_sales_previous['sales'].shift(1)
#drop nulle values
df_sales_previous = df_sales_previous.dropna()
df_sales_previous['diff_previous'] = (df_sales_previous['sales'] - df_sales_previous['diff'])
df_sales_previous.head(10)

In [ ]:
#plot the diff with the previous month
import matplotlib.pyplot as plt

# gca stands for 'get current axis'
ax = plt.gca()

df_sales_previous.plot(kind='line',x='date',y='diff_previous',ax=ax)

plt.show()

In [ ]:
#get previous month to predict the next month